In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import the libraries and the data

In [ ]:
#Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
#Import the training set
train_df=pd.read_csv("../input/black-friday/train.csv")
test_df=pd.read_csv("../input/black-friday/test.csv")

In [ ]:
#Creating a checkpoint
df=train_df.copy()

In [ ]:
#Get the data information
train_df.info()

In [ ]:
test_df.info()

In [ ]:
#Let's check the first 5 rows
train_df.head()

In [ ]:
test_df.head()

## Data Exploration and PreProcessing

In [ ]:
#Remove the User_ID field
train_df.drop('User_ID',axis=1,inplace=True)
test_df.drop('User_ID',axis=1,inplace=True)

In [ ]:
train_df.shape

In [ ]:
#Checking the basic stats from the dataset
train_df.describe()

In [ ]:
#Checking the 'Age' column unique values
train_df['Age'].unique()

In [ ]:
test_df['Age'].unique()

In [ ]:
#Mapping the data to numeric valuess
train_df['Age']=train_df['Age'].map({'0-17':0,'18-25':1, '26-35':2,'36-45':3,'46-50':4, '51-55':5, '55+':6})
test_df['Age']=test_df['Age'].map({'0-17':0,'18-25':1, '26-35':2,'36-45':3,'46-50':4, '51-55':5, '55+':6})


In [ ]:
#Check the Age column
train_df['Age'].unique()

In [ ]:
#Check the Gender Column
train_df['Gender'].unique()

In [ ]:
test_df['Gender'].unique()

In [ ]:
train_df['Gender']=train_df['Gender'].map({'F':0,'M':1})
test_df['Gender']=test_df['Gender'].map({'F':0,'M':1})

In [ ]:
#Check the Gender Column for confirmation
train_df['Gender'].unique()

In [ ]:
#Check the Marital Status Column
train_df['Marital_Status'].unique()

In [ ]:
#Get the different City Category
train_df['City_Category'].unique()

In [ ]:
#One hot encoding for City Category
city=pd.get_dummies(train_df['City_Category'],drop_first=True)

In [ ]:
city

In [ ]:
#Concatenate the City df to train_df
train_df=pd.concat([train_df,city],axis=1)

In [ ]:
#dropping the City_Category columns
train_df.drop('City_Category',axis=1,inplace=True)

In [ ]:
train_df.head()

In [ ]:
city_test=pd.get_dummies(test_df['City_Category'],drop_first=True)

In [ ]:
#Concatenate the City df to test_df
test_df=pd.concat([test_df,city_test],axis=1)

In [ ]:
#Removing Product_Id
train_df.drop('Product_ID',axis=1,inplace=True)

In [ ]:
test_df.drop('Product_ID',axis=1,inplace=True)

In [ ]:
test_df.info()

## Missing values imputation

In [ ]:
percent_missing=np.round((train_df.isna().sum()/train_df.isna().count()),3)
percent_missing.sort_values(ascending=False)

In [ ]:
#Only Product_Category_2(almost 32%) and Product_Category_3(almost 70%) has null values

In [ ]:
#Removing Product_Category_3
train_df.drop('Product_Category_3',axis=1,inplace=True)

In [ ]:
#Check the Product Category 2
train_df['Product_Category_2'].value_counts()

In [ ]:
#Impute the missing values with mode value for Product_Category_2
train_df['Product_Category_2']=train_df['Product_Category_2'].fillna(train_df['Product_Category_2'].mode()[0])

In [ ]:
#Check for missing values
train_df['Product_Category_2'].isna().sum()

In [ ]:
#Same for Test Data
percent_missing=np.round((test_df.isna().sum()/test_df.isna().count()),3)
percent_missing.sort_values(ascending=False)

In [ ]:
#Only Product_Category_2(31%) and Product_Category_3(almost 70%) has null values

In [ ]:
#Removing Product_Category_3
test_df.drop('Product_Category_3',axis=1,inplace=True)

In [ ]:
test_df['Product_Category_2']=test_df['Product_Category_2'].fillna(train_df['Product_Category_2'].mode()[0])

### Check the data types

In [ ]:
train_df.info()

In [ ]:
train_df['Stay_In_Current_City_Years'].unique()

In [ ]:
train_df['Stay_In_Current_City_Years']=train_df['Stay_In_Current_City_Years'].str.replace('+','')

In [ ]:
test_df['Stay_In_Current_City_Years']=test_df['Stay_In_Current_City_Years'].str.replace('+','')

In [ ]:
train_df['Stay_In_Current_City_Years']=train_df['Stay_In_Current_City_Years'].astype(int)
train_df['B']=train_df['B'].astype(int)
train_df['C']=train_df['C'].astype(int)

In [ ]:
test_df['Stay_In_Current_City_Years']=test_df['Stay_In_Current_City_Years'].astype(int)
test_df['B']=test_df['B'].astype(int)
test_df['C']=test_df['C'].astype(int)

In [ ]:
train_df['Product_Category_2']

In [ ]:
train_df['Product_Category_2']=train_df['Product_Category_2'].astype(float)

In [ ]:
test_df.info()

In [ ]:
#We can see this column is not present in Training dataset
test_df['City_Category']

In [ ]:
#Dropping this column
test_df.drop('City_Category',axis=1,inplace=True)

# Exploratory Data Analysis

### Gender- Who purchases more?

In [ ]:
sns.barplot('Gender','Purchase',data=train_df)

### Though the difference is not large, still men are purchasing more than women.

### Age- Which age Category is purchasing more

In [ ]:
sns.barplot('Age','Purchase',data=train_df)

#### There is not much difference of the purchasing between different Age Categories.

## Marital Status-Married People purcases more than Single?

In [ ]:
sns.barplot('Marital_Status','Purchase',data=train_df)

### There is not much difference in Purchase between Married and Single People

In [ ]:
sns.barplot('Occupation','Purchase',data=train_df)

# Model Building

In [ ]:
X=train_df.drop('Purchase',axis=1)
y=train_df['Purchase']

### Splitting into Train and Validation Set

In [ ]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.5,random_state=42)

### Random Forest Regressor

In [ ]:
rfr=RandomForestRegressor(n_estimators=150)
rfr.fit(X_train,y_train)
rfrpredict=rfr.predict(X_valid)

### Gradient Boosting Regressor

In [ ]:
gbr=GradientBoostingRegressor()
gbr.fit(X_train,y_train)
gbrpredict= gbr.predict(X_valid)

### XGB Regressor

In [ ]:
xgr=XGBRegressor()
xgr.fit(X_train,y_valid)
xgrpredict=xgr.predict(X_valid)

### Verify the RMSE code

In [ ]:
print("RMSE score for Random_Forest : ", np.sqrt(mean_squared_error(y_valid,rfrpredict)))
print("RMSE score for Gradient Boosting : ", np.sqrt(mean_squared_error(y_valid,gbrpredict)))
print("RMSE score for XG Boosting : ", np.sqrt(mean_squared_error(y_valid,xgrpredict)))

In [ ]:
finalpredict=gbr.predict(test_df)

In [ ]:
finalpredict